### This file will use the Rotational Forest Classifier from the Aeon library

In [ ]:
from _Data_Processing import *

'''
this function returns 7 objects:
	data: sets of data as X_train, X_val, X_ind, y_train, y_val, y_ind
	feature_list: list of all feature names with a column index (dict)
'''
X_train, X_val, X_ind,\
y_train, y_val, y_ind,\
feature_subsets =\
preprocess_data(
	file_name   =		'bestaset_tmp.csv'
	,indp_size  =		0.05
	,test_size  =		0.15
	,shfl_splt	=		False
	,t_start    =		570
	,t_end		=		645
	,mod_type	=		'Classification'
	,target_t	=		15
	,num_class	=		2
	,split_val	=		5
	,verbose	=		1
	,scaler		=		'Standard'
	,frmt_lstm	=		False
	,time_steps =		5
	,keep_price =		True
)

In [ ]:
from _Data_Rotating import *

'''
This function returns 1 list of rotated partitions of the training set
'''

X_train_parts = rotate_partitions(
	X				=	X_train
	,n_feat_parts	=	2
	,feat_subsets	=	feature_subsets
	,feat_part_type	=	'by_subset'
	,fraction_feats	=	0.5
	,no_feat_overlap=	False
	,feats_for_all	=	[]
	,rotation_type	=	'PCA'
	,rotation_filter=	False
	,filter_type	=	'Retention'
	,filter_value	=	1.0
    ,n_sample_parts	=	5
    ,smpl_part_type	=	'Even'
    ,sample_shuffle	=	False
)

In [ ]:
val = [1, 2, 3]

for i in val:
    i+=1
    print(i,end=' ')
print('')
print(val)